## Importing necessary libraries, Initializing the spark session and Loading the dataset :
----

In [64]:
from pyspark.sql import functions as F

from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id as mi
from pyspark.sql.functions import *
import warnings
warnings.filterwarnings('ignore')

#   Initialize the Spark session
spark = SparkSession.builder \
    .master("local") \
    .appName("Exercise2") \
    .getOrCreate()

df = spark.read.format('org.apache.spark.sql.json') \
        .load("icc_results.json")

## Creating a tally of matches won by each team and finding team that won the most number of matches :
----

In [82]:
matchoutcome = df.withColumn("A", col("scheduleentry.team1.team.fullName"))\
                    .withColumn("B", col("scheduleentry.team2.team.fullName"))

matchoutcome = matchoutcome.select('scheduleentry.matchStatus.outcome','A','B').dropna()
matchoutcome = matchoutcome.filter((matchoutcome.outcome !='N') & (matchoutcome.outcome !='D'))

matchoutcome = matchoutcome.withColumn('Winning Team', when(col('outcome')=='A', matchoutcome.A)\
                                                        .when(col('outcome')=='B',matchoutcome.B))\
                                                        .select('Winning Team')\
                                                        .groupBy('Winning Team').count()\
                                                        .orderBy(['count'], ascending = [False])
                                                        
matchoutcome.show(3)
print('The most number of matches were won by {}'.format(matchoutcome.collect()[0][0]),':',matchoutcome.collect()[0][1])

+------------+-----+
|Winning Team|count|
+------------+-----+
|       India|  279|
|   Australia|  242|
|     England|  231|
+------------+-----+
only showing top 3 rows

The most number of matches were won by India : 279
